In [2]:
# Necessary imports

%matplotlib inline
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision.io import ImageReadMode
from torchvision import transforms
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

import scipy.signal as spsig
import scipy.signal.windows as spwin

import random
import os
import librosa
import librosa.display

from PIL import Image

import csv


import pandas as pd
from torchvision import io
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch import flatten

## Generate Spectrograms
Basically convert a .wav or .mp3

In [ ]:
# Create spectrograms
def generateSpectrograms(source_dir, out_dir, window, hop):

# array of sound types available
    sound_types = ["Snare", "Trumpet", "Violin"]

    # iterate over each of the sound types
    for sound_type in sound_types:

        # establish output directories
        # TODO: Add checks that directorties exist
        sound_dir = os.fsencode(source_dir + "/" + sound_type)
        spec_dir = os.fsencode(out_dir + "/" + sound_type)

        if not os.path.exists(spec_dir):
            os.makedirs(spec_dir)
        
        # remove all existing spectrograms
        for spec_file in os.listdir(spec_dir):
            filename = os.fsdecode(spec_file)
            if filename.endswith(".png"):
                os.remove(os.path.join(spec_dir, spec_file))


        # iterate over each file (in each directory)
        for sound_file in os.listdir(sound_dir):
            filename = os.fsdecode(sound_file)

            # filter out audio files
            if filename.endswith(".wav") or filename.endswith(".mp3" or filename.endswith(".flac")):
                
                # load sound files
                # librosa is convenient because it resamples and downscales to mono automatically
                y_orig, fs = librosa.load(os.path.join(sound_dir, sound_file), mono=True, sr=48000)

                # set output time and corresponding number of samples
                output_time = 1 # seconds
                output_len = output_time * fs

                # number of samples of loaded file
                input_len = np.shape(y_orig)[0]

                # skip if loaded file is too short analyse
                if input_len < output_len:
                    print(filename, "is too short, skipping conversion")
                    continue
                
                # find a section with a high rms value
                jump = output_len//2 # spacing between sections
                rms_best = 0
                rms_best_start = 0

                # loop over sections of the sample to find the big with the best rms value
                for start in range(0, input_len - output_len, jump):
                    end = start + output_len
                    rms = np.sqrt(np.mean(np.square(y_orig[start:end])))
                    if rms > rms_best:
                        rms_best = rms
                        rms_best_start = start

                # window with best RMS value
                y = y_orig[rms_best_start:rms_best_start + output_len]

                # Compute the Short-Time Fourier Transform (STFT)
                # D = librosa.stft(y)

                # STFT calc for spectrogramss

                T_x, N = 1 / fs, output_len  # 20 Hz sampling rate for 50 s signal

                t_x = np.arange(N) * T_x  # time indexes for signal

                # SFT = spsig.ShortTimeFFT(win, hop=hop, fs=fs, mfft=16000, scale_to='psd')
                SFT = spsig.ShortTimeFFT(window, hop=hop, fs=fs, mfft=4096, scale_to='psd')

                Sx2 = SFT.spectrogram(y)  # calculate absolute square of STFT

                Sx_dB = 10 * np.log10(np.fmax(Sx2, 5e-9))  # limit range to ~-83dB

                fig1, ax1 = plt.subplots(figsize=(6., 4.))  # enlarge plot a bit
                im1 = ax1.imshow(Sx_dB, origin='lower', aspect='auto', extent=SFT.extent(N), cmap='gray')


                output_file = os.path.join(spec_dir, os.fsencode(os.path.splitext(filename)[0] + ".png"))
                plt.axis("off")

                plt.savefig(output_file, bbox_inches='tight', pad_inches=0)        
                plt.close()

rectangular = np.ones(1024)

generateSpectrograms("../Training-data/Sounds", "../Training-data/Rectangular", rectangular, 512)
generateSpectrograms("../Test-data/Sounds", "../Test-data/Rectangular", rectangular, 512)

hamming = spwin.hamming(1024)

generateSpectrograms("../Training-data/Sounds", "../Training-data/Hamming", hamming, 512)
generateSpectrograms("../Test-data/Sounds", "../Test-data/Hamming", hamming, 512)

blackman = spwin.blackman(1024)

generateSpectrograms("../Training-data/Sounds", "../Training-data/Blackman", blackman, 512)
generateSpectrograms("../Test-data/Sounds", "../Test-data/Blackman", blackman, 512)



[src/libmpg123/id3.c:process_extra():681] error: No extra frame text / valid description?
[src/libmpg123/id3.c:process_extra():681] error: No extra frame text / valid description?
[src/libmpg123/id3.c:process_extra():681] error: No extra frame text / valid description?
[src/libmpg123/id3.c:process_extra():681] error: No extra frame text / valid description?
[src/libmpg123/id3.c:process_extra():681] error: No extra frame text / valid description?
[src/libmpg123/id3.c:process_extra():681] error: No extra frame text / valid description?
[src/libmpg123/id3.c:process_extra():681] error: No extra frame text / valid description?
[src/libmpg123/id3.c:process_extra():681] error: No extra frame text / valid description?
[src/libmpg123/id3.c:process_extra():681] error: No extra frame text / valid description?
[src/libmpg123/id3.c:process_extra():681] error: No extra frame text / valid description?
[src/libmpg123/id3.c:process_extra():681] error: No extra frame text / valid description?
[src/libmp

In [ ]:
# Create arrays of ground truth data

def generateGroundTruthCSV(specdir, groundTruthCSVName):

    sound_types = ["Snare", "Trumpet", "Violin"]

    data = []

    for sound_type in sound_types:

        # establish output directories
        # TODO: Add checks that directorties exist
        spec_dir = os.fsencode(specdir + "/" + sound_type)
        
        # remove all existing spectrograms
        for spec_file in os.listdir(spec_dir):
            
            filename = os.fsdecode(spec_file)
            
            if filename.endswith(".png"):

                data.append([os.fsdecode(os.path.join(spec_dir, spec_file)), sound_type])

    key = lambda row: row[0] 

    data.sort(key=lambda item: item[0])

    with open(groundTruthCSVName, 'w+') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerows(data)

generateGroundTruthCSV("../Training-data/Rectangular", "../Training-data/rectangular.csv")
generateGroundTruthCSV("../Training-data/Hamming", "../Training-data/hamming.csv")
generateGroundTruthCSV("../Training-data/Blackman", "../Training-data/blackman.csv")

generateGroundTruthCSV("../Test-data/Rectangular", "../Test-data/rectangular.csv")
generateGroundTruthCSV("../Test-data/Hamming", "../Test-data/hamming.csv")
generateGroundTruthCSV("../Test-data/Blackman", "../Test-data/blackman.csv")

## Define classes

In [3]:
classes = ("Snare", "Trumpet", "Violin")

## Create dataset class


In [55]:


class InstrumentsDataset(Dataset):
    """Instruments dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.instrument_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.instrument_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.instrument_frame.iloc[idx, 0])
        # image = io.read_image(img_name, mode=ImageReadMode.GRAY)
        # image = io.read_image(img_name)
        image = Image.open(img_name)



        if self.transform:
            image = self.transform(image)

        
        instrument = self.instrument_frame.iloc[idx, 1]
        instrument_id = classes.index(instrument)

        return image, instrument_id


C:\Users\pilla\AppData\Local\Temp\ipykernel_24088\2176705582.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Create model

In [78]:
class Model(nn.Module):
    def __init__(self, numChannels,numClasses):
  
        super(Model, self).__init__()
        #First layer (convolutional)
        self.conv1 = nn.Conv2d(numChannels, 6, 3)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(2, 2)

        # Second layer (also convolutional)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(2, 2)

        # an affine operation: y = Wx + b
        self.fc1 = nn.LazyLinear(120)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.LazyLinear(numClasses)
        self.logsoftmax = nn.LogSoftmax(dim=1)


    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        # Max pooling over a (2, 2) window
        x = flatten(x, start_dim=1, end_dim=-1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        x = self.logsoftmax(x)
        return x

## Create datasets and dataloaders

In [88]:
# transform = transforms.Compose(
#     [transforms.Grayscale(),
#      transforms.ToTensor(),
#      transforms.Normalize((0.5), (0.5)),
#      transforms.Resize((64, 64))])

trans = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor()
    # transforms.Resize((64, 64))
    ])

train_val_dataset = InstrumentsDataset(csv_file = "../Training-data/blackman.csv", root_dir = ".", transform=trans)
test_dataset = InstrumentsDataset(csv_file = "../Test-data/blackman.csv", root_dir = ".", transform = trans)

train_size = int(0.8 * len(train_val_dataset))
val_size = len(train_val_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_val_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16)
val_loader = DataLoader(val_dataset, batch_size=16)

test_loader = DataLoader(test_dataset, batch_size=16)





In [59]:
image, instrument_id = train_dataset[0]

print(image)
print(image.shape)

tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.2908, 0.3193, 0.3050,  ..., 0.3779, 0.3728, 0.3181],
         [0.4043, 0.4170, 0.4213,  ..., 0.4659, 0.4229, 0.3139],
         [0.7088, 0.6894, 0.6708,  ..., 0.5156, 0.5531, 0.4057]]])
torch.Size([1, 64, 64])


## Go and run

In [89]:
model = Model(1, len(classes)) # 1 channel for grayscale at this stage
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

def evaluate(model, loader):
    model.eval()
    # initialise evaluation parameters
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad(): # evaluating so don't produce gradients
        for data in loader:
            inputs, labels = data # get data from dataloader
            outputs = model(inputs) # predict outputs
            loss = criterion(outputs, labels) # calculate current loss
            _, predicted = torch.max(outputs.data, 1) # calculate predicted data
            total += labels.size(0) # total number of labels in the current batch
            correct += (predicted == labels).sum().item() # number of labels that are correct
            
            running_loss += loss.item() # loss? not 100% sure
        
    # Return mean loss, accuracy
    return running_loss / len(loader), correct / total

loss_history = {
        'train_loss': [],
        'val_loss': []
}


for epoch in range(50):  # loop over the dataset multiple times
    print("Starting Epoch: {}".format(epoch+1))
    
    running_loss = 0.0
    
    for i, data in enumerate(train_loader, 0):
        model.train()
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 5 == 4:    # print every 5 mini-batches
            mean_loss = running_loss / 100
            loss_history['train_loss'].append(mean_loss)
            print('# mini-batch {}\ntrain loss: {}'.format(
                  i + 1, mean_loss))
            running_loss = 0.0
            
            # evaluate on validation dataset
            mean_loss, val_acc = evaluate(model, val_loader)
            loss_history['val_loss'].append(mean_loss)
                  
            print("validation loss: {} validation accuracy: {}\n".format(mean_loss, val_acc))

print('Finished Training')


# Notes for tomorrow:
# Check what type of data is being input / output
# Look at a tutorial on custom pytorch image recognition?
# https://medium.com/analytics-vidhya/implementing-cnn-in-pytorch-with-custom-dataset-and-transfer-learning-1864daac14cc
# https://glassboxmedicine.com/2021/02/06/designing-custom-2d-and-3d-cnns-in-pytorch-tutorial-with-code/

/home/robert/venv/lib/python3.11/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Starting Epoch: 1
# mini-batch 5
train loss: 0.2046717071533203
validation loss: 1.2205586830774944 validation accuracy: 0.5128205128205128

# mini-batch 10
train loss: 0.06070639669895172
validation loss: 0.9739670753479004 validation accuracy: 0.38461538461538464

Starting Epoch: 2
# mini-batch 5
train loss: 0.03966433107852936
validation loss: 0.6906323035558065 validation accuracy: 0.6923076923076923

# mini-batch 10
train loss: 0.03739737987518311
validation loss: 0.6232115228970846 validation accuracy: 0.5897435897435898

Starting Epoch: 3
# mini-batch 5
train loss: 0.03158049881458282
validation loss: 0.538288950920105 validation accuracy: 0.717948717948718

# mini-batch 10
train loss: 0.02888959139585495
validation loss: 0.5607175926367441 validation accuracy: 0.717948717948718

Starting Epoch: 4
# mini-batch 5
train loss: 0.024217396676540374
validation loss: 0.5731888711452484 validation accuracy: 0.6153846153846154

# mini-batch 10
train loss: 0.024933967888355255
validation

KeyboardInterrupt: 